In [1]:
import datetime
import pandas as pd
import numpy as np

Baca Data

In [2]:
Holidays_df = pd.read_csv('./Dataset/Holidays.csv', low_memory=False)
Stock_df = pd.read_csv('./Dataset/Clean/Stock_clean.csv',sep=",",low_memory=False)
Order_df = pd.read_csv('./Dataset/Clean/Order_clean.csv',sep=",",low_memory=False)
OrderItem_df = pd.read_csv('./Dataset/Clean/OrderItem_clean.csv',sep=",",low_memory=False)
Branch_df = pd.read_csv('./Dataset/Clean/Branch_clean.csv',sep=",",low_memory=False)
Item_df = pd.read_csv('./Dataset/Clean/Item_clean.csv',sep=",",low_memory=False)

Merge Data

In [3]:
Merged_df = pd.merge(OrderItem_df, Order_df, on='POSNo')
Daily_df = Merged_df.groupby(['BranchID','ItemID','POSDate']).agg({'Qty': 'sum', 'Price': 'last'}).sort_values(by=['BranchID','ItemID']).reset_index()
Daily_df = Daily_df.rename(columns={'Qty': 'Qty_sum', 'Price': 'PricePerItem'})
Daily_df = Daily_df.astype({'POSDate': 'datetime64[ns]'})
Daily_df

,BranchID,ItemID,POSDate,Qty_sum,PricePerItem
0,BA15019,00003408,2024-01-02,8,198000
1,BA15019,00003408,2024-01-05,1,198000
2,BA15019,00003408,2024-01-08,4,198000
3,BA15019,00003408,2024-01-09,1,198000
4,BA15019,00003408,2024-01-12,1,198000
...,...,...,...,...,...
2316833,YK42089,00001874,2024-05-01,1,15500
2316834,YK42089,00001874,2024-05-03,12,15500
2316835,YK42089,00001874,2024-05-04,13,15500
2316836,YK42089,00001874,2024-05-05,22,15500


In [4]:
# Performed 1 aggregation grouped on columns: 'BranchID', 'ItemID'
Prices_df = Daily_df.groupby(['BranchID', 'ItemID']).agg(PricePerItem=('PricePerItem','last')).reset_index()
Prices_df

,BranchID,ItemID,PricePerItem
0,BA15019,00003408,198000
1,BA15019,12500006,100000
2,BA15019,12500007,260000
3,BD32002,00000047,20000
4,BD32002,00000065,20000
...,...,...,...
83946,YK42088,PRM-2303000285,9900
83947,YK42088,PRM-2303000286,9900
83948,YK42088,PRM-2303000288,9900
83949,YK42088,PRM-2303000289,9900


Complete Branch & ID

In [5]:
Branches = pd.DataFrame({'BranchID': Merged_df['BranchID'].unique()})
Branches

,BranchID
0,JK32233
1,SM42081
2,SB52067
3,BD32177
4,BK32046
...,...
188,JK35072
189,JK32025
190,SK32206
191,JK32012


In [6]:
Items = pd.DataFrame({'ItemID': Merged_df['ItemID'].unique()})
Items

,ItemID
0,00002809
1,00001594
2,00101167
3,00000065
4,00037168
...,...
1917,12125049
1918,01092133
1919,01092135
1920,00002626


In [7]:
Branch_df = pd.merge(Branch_df, Branches, on='BranchID', how='outer')
Branch_df

,BranchID,BranchName
0,BA15019,REG 1 - SPBU BANDA ACEH SUDIRMAN
1,BA61006,REG 6 - BANJARBARU DC LPG
2,BB31018,REG 3 - KOTA BANJAR DC LPG
3,BC61010,REG 6 - BALANGAN DC LPG
4,BD31029,REG 3 - BANDUNG DC LPG NPSO
...,...,...
388,YK42061,REG 4 - YOGYAKARTA WONOSARI
389,YK42088,REG 4 - YOGYAKARTA COKROAMINOTO
390,YK42089,REG 4 - YOGYAKARTA BIMASAKTI
391,YK42092,REG 4 - EVENT PRAMBANAN JAZZ 2023


In [8]:
Item_df = pd.merge(Item_df, Items, on='ItemID', how='outer')
Item_df

,ItemID,ItemName,BaseUOM
0,00000004,SOFTEX SUPER DELUX MAXI 8,PCS
1,00000005,SOFTEX MAXI WINGS 8,PCS
2,00000006,NISSIN WAFER MILK 120 GR,PCS
3,00000007,CITRA H&B LOT TEH HIJAU 60 ML,PCS
4,00000008,DYNASTY TISSUE TOILET TOD 01,PCS
...,...,...,...
17154,PRM-2404000016,NaN,NaN
17155,PRM-2404000066,NaN,NaN
17156,PRM-2404000067,NaN,NaN
17157,PRM-2404000068,NaN,NaN


In [9]:
Branch_df['Branch'] = Branch_df.apply(
    lambda row: row['BranchID']
    if pd.isna(row['BranchName']) 
    else f'{row['BranchID']} - {row['BranchName']}', 
    axis=1)
Branch_df = Branch_df.sort_values(by='BranchID')
Branch_df

,BranchID,BranchName,Branch
0,BA15019,REG 1 - SPBU BANDA ACEH SUDIRMAN,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN
1,BA61006,REG 6 - BANJARBARU DC LPG,BA61006 - REG 6 - BANJARBARU DC LPG
2,BB31018,REG 3 - KOTA BANJAR DC LPG,BB31018 - REG 3 - KOTA BANJAR DC LPG
3,BC61010,REG 6 - BALANGAN DC LPG,BC61010 - REG 6 - BALANGAN DC LPG
4,BD31029,REG 3 - BANDUNG DC LPG NPSO,BD31029 - REG 3 - BANDUNG DC LPG NPSO
...,...,...,...
388,YK42061,REG 4 - YOGYAKARTA WONOSARI,YK42061 - REG 4 - YOGYAKARTA WONOSARI
389,YK42088,REG 4 - YOGYAKARTA COKROAMINOTO,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO
390,YK42089,REG 4 - YOGYAKARTA BIMASAKTI,YK42089 - REG 4 - YOGYAKARTA BIMASAKTI
391,YK42092,REG 4 - EVENT PRAMBANAN JAZZ 2023,YK42092 - REG 4 - EVENT PRAMBANAN JAZZ 2023


In [10]:
Item_df['Item'] = Item_df.apply(
    lambda row: row['ItemID']
    if pd.isna(row['ItemName']) 
    else f'{row['ItemID']} - {row['ItemName']} ({row['BaseUOM']})', 
    axis=1)
Item_df = Item_df.sort_values(by='ItemID')
Item_df

,ItemID,ItemName,BaseUOM,Item
0,00000004,SOFTEX SUPER DELUX MAXI 8,PCS,00000004 - SOFTEX SUPER DELUX MAXI 8 (PCS)
1,00000005,SOFTEX MAXI WINGS 8,PCS,00000005 - SOFTEX MAXI WINGS 8 (PCS)
2,00000006,NISSIN WAFER MILK 120 GR,PCS,00000006 - NISSIN WAFER MILK 120 GR (PCS)
3,00000007,CITRA H&B LOT TEH HIJAU 60 ML,PCS,00000007 - CITRA H&B LOT TEH HIJAU 60 ML (PCS)
4,00000008,DYNASTY TISSUE TOILET TOD 01,PCS,00000008 - DYNASTY TISSUE TOILET TOD 01 (PCS)
...,...,...,...,...
17154,PRM-2404000016,NaN,NaN,PRM-2404000016
17155,PRM-2404000066,NaN,NaN,PRM-2404000066
17156,PRM-2404000067,NaN,NaN,PRM-2404000067
17157,PRM-2404000068,NaN,NaN,PRM-2404000068


Input Datetime

In [11]:
# Date_now = datetime.datetime.now()
Date_now = max(Daily_df['POSDate'])
prediction_days = 7

Days = []
Months = []
Years = []
# Menghitung tanggal ke depan
for i in range(prediction_days):
    new_date = Date_now + datetime.timedelta(days=i+1)
    Days.append(new_date.day)
    Months.append(new_date.month)
    Years.append(new_date.year)

Days = np.array(Days)
Months = np.array(Months)
Years = np.array(Years)

Function untuk Mendapatkan DF dengan Format yang Sesuai

In [12]:
def affected_by_holiday(date, holidays):
    date = date.to_datetime64()
    holidays_date = pd.to_datetime(holidays['Date'])
    days_to_holiday = date - holidays_date
    number_of_holidays = len(days_to_holiday)
    affected = [False for i in range(number_of_holidays)]
    
    for i in range(number_of_holidays):
        days = days_to_holiday[i]
        preparation_days = pd.to_timedelta(holidays['Preparation'][i], unit='D')
        aftermath_days = pd.to_timedelta(holidays['Aftermath'][i], unit='D')
        if (days < pd.Timedelta(0)) & (days >= -preparation_days):
            affected[i] = True
        elif (days > pd.Timedelta(0)) & (days <= aftermath_days):
            affected[i] = True
        else:
            affected[i] = holidays['DDay'][i]

    return any(affected)

In [13]:
def convert_input(day, month, year, holidays, prices):
    # Output Dataframe
    date = datetime.date(year, month, day)
    date_time = {'POSDate': [date],
                 'POSDay': [day],
                 'POSMonth': [month],
                 'POSYear': [year],
                 'Discount': False}
    input_data = pd.DataFrame(date_time).astype({'POSDay': 'int64','POSMonth': 'int64','POSYear': 'int64'})
    input_data['POSDate'] = pd.to_datetime(input_data['POSDate'])
    input_data['AffectedByHoliday'] = input_data['POSDate'].apply(lambda date: affected_by_holiday(date, holidays))
    # Merging data
    input_data = pd.concat([input_data, prices])
    input_data = input_data.fillna(method='ffill')
    # Membuang data pertama (dummy)
    input_data = input_data.iloc[1:]
    return input_data

Prediksi Data

In [14]:
# Membuat Dataframe yang digunakan sebagai input yang nanti akan digunakan
Input_Daily_df = pd.DataFrame()
for i in range(prediction_days):
    Input_Daily_df = pd.concat([Input_Daily_df, convert_input(Days[i], Months[i], Years[i], Holidays_df, Prices_df)], ignore_index=True)
# Menyimpan panjang data untuk keperluan manipulasi
Input_Daily_length = len(Input_Daily_df)
Input_Daily_df = Input_Daily_df.astype({'POSDay': 'int64', 'POSMonth': 'int64', 'POSYear': 'int64', 'PricePerItem': 'int64'})
Input_Daily_df

C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: Downcasting object

,POSDate,POSDay,POSMonth,POSYear,Discount,AffectedByHoliday,BranchID,ItemID,PricePerItem
0,2024-05-09,9,5,2024,False,False,BA15019,00003408,198000
1,2024-05-09,9,5,2024,False,False,BA15019,12500006,100000
2,2024-05-09,9,5,2024,False,False,BA15019,12500007,260000
3,2024-05-09,9,5,2024,False,False,BD32002,00000047,20000
4,2024-05-09,9,5,2024,False,False,BD32002,00000065,20000
...,...,...,...,...,...,...,...,...,...
587652,2024-05-15,15,5,2024,False,False,YK42088,PRM-2303000285,9900
587653,2024-05-15,15,5,2024,False,False,YK42088,PRM-2303000286,9900
587654,2024-05-15,15,5,2024,False,False,YK42088,PRM-2303000288,9900
587655,2024-05-15,15,5,2024,False,False,YK42088,PRM-2303000289,9900


In [15]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
model = xgb.Booster()
model.load_model('./Model/xgboost_model_daily.bst')

X = Input_Daily_df[['BranchID', 'ItemID', 'POSDay', 'POSMonth', 'POSYear', 'PricePerItem', 'Discount', 'AffectedByHoliday']]
# Use sparse matrix
encoder_D = OneHotEncoder(sparse_output=True, handle_unknown='ignore')  # Using sparse output to save memory
X_encoded = encoder_D.fit_transform(X[['BranchID', 'ItemID']])

# Drop original columns earlier to free memory
X.drop(columns=['BranchID', 'ItemID'], inplace=True)

# Convert the sparse matrix to DataFrame
X_encoded = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=encoder_D.get_feature_names_out(['BranchID', 'ItemID']))

# Concatenate with original DataFrame
X = pd.concat([X, X_encoded], axis=1)

# Display DataFrame
X

C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\2033632209.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['BranchID', 'ItemID'], inplace=True)


,POSDay,POSMonth,POSYear,PricePerItem,Discount,AffectedByHoliday,BranchID_BA15019,BranchID_BD32002,BranchID_BD32007,BranchID_BD32008,...,ItemID_PRM-2404000008,ItemID_PRM-2404000009,ItemID_PRM-2404000010,ItemID_PRM-2404000011,ItemID_PRM-2404000012,ItemID_PRM-2404000016,ItemID_PRM-2404000066,ItemID_PRM-2404000067,ItemID_PRM-2404000068,ItemID_PRM-2404000069
0,9,5,2024,198000,False,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,5,2024,100000,False,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,5,2024,260000,False,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,5,2024,20000,False,False,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,5,2024,20000,False,False,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587652,15,5,2024,9900,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
587653,15,5,2024,9900,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
587654,15,5,2024,9900,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
587655,15,5,2024,9900,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X = xgb.DMatrix(X)
predictions = model.predict(X)
predictions = np.rint(predictions).astype("int64")

Memasukan Hasil Prediksi ke Input_df

In [17]:
num_inputs = (Input_Daily_length//prediction_days)
Output_Daily_df = Input_Daily_df.iloc[:num_inputs]
Output_Daily_df = pd.merge(Output_Daily_df, Branch_df, on='BranchID', how='left')
Output_Daily_df = pd.merge(Output_Daily_df, Item_df, on='ItemID', how='left')
Output_Daily_df['POSDate'] = pd.to_datetime(Date_now)
Output_Daily_df = Output_Daily_df.rename(columns={'POSDate': 'LastDate'})
cols = ['BranchID','Branch','ItemID','Item','LastDate','PricePerItem','Discount','AffectedByHoliday']
Output_Daily_df = Output_Daily_df[cols]

for idx in range(prediction_days):
    Output_Daily_df[f'H+{idx+1}'] = predictions[(idx*num_inputs):((idx+1)*num_inputs)]
    Output_Daily_df[f'H+{idx+1}'] = Output_Daily_df[f'H+{idx+1}'].apply(lambda Qty: max(0,Qty))
Output_Daily_df

,BranchID,Branch,ItemID,Item,LastDate,PricePerItem,Discount,AffectedByHoliday,H+1,H+2,H+3,H+4,H+5,H+6,H+7
0,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,00003408,00003408 - GAS 12 KG (REFILL) (TBG),2024-05-08,198000,False,False,3,3,3,3,3,3,3
1,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,12500006,"12500006 - GAS 5,5KG (REFILL) (TBG)",2024-05-08,100000,False,False,1,1,1,1,1,1,1
2,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,12500007,"12500007 - TABUNG GAS 5,5KG (TBG)",2024-05-08,260000,False,False,0,0,0,0,0,0,0
3,BD32002,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000047,00000047 - FISHERMAN'S F LEMON 25 GR (PCS),2024-05-08,20000,False,False,1,1,1,1,1,1,1
4,BD32002,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000065,00000065 - FISHERMAN'S F EXTRA STRONG 25GR (PCS),2024-05-08,20000,False,False,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83946,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000285,PRM-2303000285,2024-05-08,9900,False,False,1,1,1,1,1,1,1
83947,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000286,PRM-2303000286,2024-05-08,9900,False,False,1,1,1,1,1,1,1
83948,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000288,PRM-2303000288,2024-05-08,9900,False,False,1,1,1,1,1,1,1
83949,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000289,PRM-2303000289,2024-05-08,9900,False,False,1,1,1,1,1,1,1


Menambah Kolom Stock_overtime 

In [18]:
Output_Daily_df = pd.merge(Output_Daily_df,Stock_df,how='left',on=['BranchID','ItemID']).fillna(0).astype({'QtyAvailable': 'int64'})
for idx in range(prediction_days):
    column_name = f'QtyAvailable+{idx+1}'
    if idx == 0:
        Output_Daily_df = Output_Daily_df.assign(
            **{column_name: Output_Daily_df['QtyAvailable'] - Output_Daily_df[f'H+{idx+1}']}
        )
    else:
        prev_column_name = f'QtyAvailable+{idx}'
        Output_Daily_df = Output_Daily_df.assign(
            **{column_name: Output_Daily_df[prev_column_name] - Output_Daily_df[f'H+{idx+1}']}
        )
Output_Daily_df = Output_Daily_df.drop(columns=['BranchID','ItemID'])
Output_Daily_df

,Branch,Item,LastDate,PricePerItem,Discount,AffectedByHoliday,H+1,H+2,H+3,H+4,...,H+6,H+7,QtyAvailable,QtyAvailable+1,QtyAvailable+2,QtyAvailable+3,QtyAvailable+4,QtyAvailable+5,QtyAvailable+6,QtyAvailable+7
0,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,00003408 - GAS 12 KG (REFILL) (TBG),2024-05-08,198000,False,False,3,3,3,3,...,3,3,0,-3,-6,-9,-12,-15,-18,-21
1,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,"12500006 - GAS 5,5KG (REFILL) (TBG)",2024-05-08,100000,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
2,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,"12500007 - TABUNG GAS 5,5KG (TBG)",2024-05-08,260000,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000047 - FISHERMAN'S F LEMON 25 GR (PCS),2024-05-08,20000,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
4,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000065 - FISHERMAN'S F EXTRA STRONG 25GR (PCS),2024-05-08,20000,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83946,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000285,2024-05-08,9900,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
83947,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000286,2024-05-08,9900,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
83948,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000288,2024-05-08,9900,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7
83949,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000289,2024-05-08,9900,False,False,1,1,1,1,...,1,1,0,-1,-2,-3,-4,-5,-6,-7


Menyimpan .CSV

In [19]:
Output_Daily_df.to_csv("Output_daily.csv",index=False)

Weekly

In [20]:
weekday = Date_now.isocalendar().weekday
if weekday != 7:
    last_date = Date_now - datetime.timedelta(days=weekday)
else:
    last_date = Date_now

prediction_weeks = 4

Days = []
Months = []
Years = []
# Menghitung tanggal ke depan
for i in range(prediction_weeks*7):
    new_date = last_date + datetime.timedelta(days=i+1)
    Days.append(new_date.day)
    Months.append(new_date.month)
    Years.append(new_date.year)

Days = np.array(Days)
Months = np.array(Months)
Years = np.array(Years)

In [21]:
# Membuat Dataframe yang digunakan sebagai input yang nanti akan digunakan
Input_Weekly_df = pd.DataFrame()
for i in range(prediction_weeks*7):
    Input_Weekly_df = pd.concat([Input_Weekly_df, convert_input(Days[i], Months[i], Years[i], Holidays_df, Prices_df)], ignore_index=True)
Input_Weekly_df['POSWeek'] = Input_Weekly_df['POSDate'].dt.isocalendar().week
Input_Weekly_df['POSYear'] = Input_Weekly_df['POSDate'].dt.isocalendar().year
Input_Weekly_df = Input_Weekly_df.groupby(['BranchID','ItemID','POSYear','POSWeek']).agg({'PricePerItem': 'last', 'Discount': 'any', 'AffectedByHoliday': 'any'}).reset_index()
Input_Weekly_df['POSDate'] = Input_Weekly_df.apply(lambda row: datetime.datetime.fromisocalendar(row['POSYear'], row['POSWeek'], 1), axis=1)
Input_Weekly_df = Input_Weekly_df.sort_values(by=['POSDate','BranchID','ItemID'])
Input_Weekly_df = Input_Weekly_df.astype({'PricePerItem': 'int64'})
cols = ['BranchID', 'ItemID', 'POSWeek', 'POSYear', 'PricePerItem', 'Discount', 'AffectedByHoliday']
Input_Weekly_df = Input_Weekly_df[cols]

# Menyimpan panjang data untuk keperluan manipulasi
Input_Weekly_length = len(Input_Weekly_df)
Input_Weekly_df

C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill')
C:\Users\T Dustin P\AppData\Local\Temp\ipykernel_8828\147576251.py:14: FutureWarning: Downcasting object

,BranchID,ItemID,POSWeek,POSYear,PricePerItem,Discount,AffectedByHoliday
0,BA15019,00003408,19,2024,198000,False,False
4,BA15019,12500006,19,2024,100000,False,False
8,BA15019,12500007,19,2024,260000,False,False
12,BD32002,00000047,19,2024,20000,False,False
16,BD32002,00000065,19,2024,20000,False,False
...,...,...,...,...,...,...,...
335787,YK42088,PRM-2303000285,22,2024,9900,False,False
335791,YK42088,PRM-2303000286,22,2024,9900,False,False
335795,YK42088,PRM-2303000288,22,2024,9900,False,False
335799,YK42088,PRM-2303000289,22,2024,9900,False,False


In [22]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
model = xgb.Booster()
model.load_model('./Model/xgboost_model_weekly.bst')

X = Input_Weekly_df[['BranchID', 'ItemID', 'POSWeek', 'POSYear', 'PricePerItem', 'Discount', 'AffectedByHoliday']]
# Use sparse matrix
encoder_D = OneHotEncoder(sparse_output=True, handle_unknown='ignore')  # Using sparse output to save memory
X_encoded = encoder_D.fit_transform(X[['BranchID', 'ItemID']])

# Drop original columns earlier to free memory
X.drop(columns=['BranchID', 'ItemID'], inplace=True)

# Convert the sparse matrix to DataFrame
X_encoded = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=encoder_D.get_feature_names_out(['BranchID', 'ItemID']))

# Concatenate with original DataFrame
X = pd.concat([X, X_encoded], axis=1)

# Display DataFrame
X

,POSWeek,POSYear,PricePerItem,Discount,AffectedByHoliday,BranchID_BA15019,BranchID_BD32002,BranchID_BD32007,BranchID_BD32008,BranchID_BD32033,...,ItemID_PRM-2404000008,ItemID_PRM-2404000009,ItemID_PRM-2404000010,ItemID_PRM-2404000011,ItemID_PRM-2404000012,ItemID_PRM-2404000016,ItemID_PRM-2404000066,ItemID_PRM-2404000067,ItemID_PRM-2404000068,ItemID_PRM-2404000069
0,19,2024,198000,False,False,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,19,2024,100000,False,False,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,19,2024,260000,False,False,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,19,2024,20000,False,False,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,19,2024,20000,False,False,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335787,22,2024,9900,False,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335791,22,2024,9900,False,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335795,22,2024,9900,False,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335799,22,2024,9900,False,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X = xgb.DMatrix(X)
predictions = model.predict(X)
predictions = np.rint(predictions).astype("int64")

In [32]:
num_inputs = (Input_Weekly_length//prediction_weeks)
Output_Weekly_df = Input_Weekly_df.iloc[:num_inputs]
Output_Weekly_df = pd.merge(Output_Weekly_df, Branch_df, on='BranchID', how='left')
Output_Weekly_df = pd.merge(Output_Weekly_df, Item_df, on='ItemID', how='left')
iso_last_date = datetime.datetime.isocalendar(last_date)
Output_Weekly_df['LastWeek'] = Output_Weekly_df.apply(lambda row: iso_last_date.week, axis=1)
Output_Weekly_df['LastWeekYear'] = Output_Weekly_df.apply(lambda row: iso_last_date.year, axis=1)
Output_Weekly_df['LastWeekStart'] = Output_Weekly_df.apply(lambda row: last_date - datetime.timedelta(days=6), axis=1)
Output_Weekly_df['LastWeekEnd'] = Output_Weekly_df.apply(lambda row: last_date, axis=1)
cols = ['BranchID','Branch','ItemID','Item','LastWeek','LastWeekYear','LastWeekStart','LastWeekEnd','PricePerItem','Discount','AffectedByHoliday']
Output_Weekly_df = Output_Weekly_df[cols]

for idx in range(prediction_weeks):
    Output_Weekly_df[f'W+{idx+1}'] = predictions[(idx*num_inputs):((idx+1)*num_inputs)]
    Output_Weekly_df[f'W+{idx+1}'] = Output_Weekly_df[f'W+{idx+1}'].apply(lambda Qty: max(0,Qty))
Output_Weekly_df

,BranchID,Branch,ItemID,Item,LastWeek,LastWeekYear,LastWeekStart,LastWeekEnd,PricePerItem,Discount,AffectedByHoliday,W+1,W+2,W+3,W+4
0,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,00003408,00003408 - GAS 12 KG (REFILL) (TBG),18,2024,2024-04-29,2024-05-05,198000,False,False,12,3,0,0
1,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,12500006,"12500006 - GAS 5,5KG (REFILL) (TBG)",18,2024,2024-04-29,2024-05-05,100000,False,False,0,0,0,0
2,BA15019,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,12500007,"12500007 - TABUNG GAS 5,5KG (TBG)",18,2024,2024-04-29,2024-05-05,260000,False,False,0,0,0,0
3,BD32002,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000047,00000047 - FISHERMAN'S F LEMON 25 GR (PCS),18,2024,2024-04-29,2024-05-05,20000,False,False,1,1,1,1
4,BD32002,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000065,00000065 - FISHERMAN'S F EXTRA STRONG 25GR (PCS),18,2024,2024-04-29,2024-05-05,20000,False,False,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83946,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000285,PRM-2303000285,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1
83947,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000286,PRM-2303000286,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1
83948,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000288,PRM-2303000288,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1
83949,YK42088,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000289,PRM-2303000289,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1


In [33]:
Output_Weekly_df = pd.merge(Output_Weekly_df,Stock_df,how='left',on=['BranchID','ItemID']).fillna(0).astype({'QtyAvailable': 'int64'})
for idx in range(prediction_weeks):
    column_name = f'QtyAvailable+{idx+1}'
    if idx == 0:
        Output_Weekly_df = Output_Weekly_df.assign(
            **{column_name: Output_Weekly_df['QtyAvailable'] - Output_Weekly_df[f'W+{idx+1}']}
        )
    else:
        prev_column_name = f'QtyAvailable+{idx}'
        Output_Weekly_df = Output_Weekly_df.assign(
            **{column_name: Output_Weekly_df[prev_column_name] - Output_Weekly_df[f'W+{idx+1}']}
        )
Output_Weekly_df = Output_Weekly_df.drop(columns=['BranchID','ItemID'])
Output_Weekly_df

,Branch,Item,LastWeek,LastWeekYear,LastWeekStart,LastWeekEnd,PricePerItem,Discount,AffectedByHoliday,W+1,W+2,W+3,W+4,QtyAvailable,QtyAvailable+1,QtyAvailable+2,QtyAvailable+3,QtyAvailable+4
0,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,00003408 - GAS 12 KG (REFILL) (TBG),18,2024,2024-04-29,2024-05-05,198000,False,False,12,3,0,0,0,-12,-15,-15,-15
1,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,"12500006 - GAS 5,5KG (REFILL) (TBG)",18,2024,2024-04-29,2024-05-05,100000,False,False,0,0,0,0,0,0,0,0,0
2,BA15019 - REG 1 - SPBU BANDA ACEH SUDIRMAN,"12500007 - TABUNG GAS 5,5KG (TBG)",18,2024,2024-04-29,2024-05-05,260000,False,False,0,0,0,0,0,0,0,0,0
3,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000047 - FISHERMAN'S F LEMON 25 GR (PCS),18,2024,2024-04-29,2024-05-05,20000,False,False,1,1,1,1,0,-1,-2,-3,-4
4,BD32002 - REG 3 - BANDUNG UJUNG BERUNG 1,00000065 - FISHERMAN'S F EXTRA STRONG 25GR (PCS),18,2024,2024-04-29,2024-05-05,20000,False,False,1,1,1,1,0,-1,-2,-3,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83946,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000285,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1,0,-1,-2,-3,-4
83947,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000286,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1,0,-1,-2,-3,-4
83948,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000288,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1,0,-1,-2,-3,-4
83949,YK42088 - REG 4 - YOGYAKARTA COKROAMINOTO,PRM-2303000289,18,2024,2024-04-29,2024-05-05,9900,False,False,1,1,1,1,0,-1,-2,-3,-4


In [34]:
Output_Weekly_weeks = pd.DataFrame()
for i in range(prediction_weeks):
    Output_Weekly_weeks.loc[i,'WeekEndDate'] = last_date + datetime.timedelta(days=7*(i+1))
Output_Weekly_weeks['WeekStartDate'] = Output_Weekly_weeks.apply(lambda row: row['WeekEndDate'] - datetime.timedelta(days=6), axis=1)
Output_Weekly_weeks['WeekNumber'] = Output_Weekly_weeks.apply(lambda row: row['WeekEndDate'].isocalendar().week, axis=1)
Output_Weekly_weeks['Year'] = Output_Weekly_weeks.apply(lambda row: row['WeekEndDate'].isocalendar().year, axis=1)
Output_Weekly_weeks = Output_Weekly_weeks[['WeekNumber', 'Year', 'WeekStartDate', 'WeekEndDate']]
Output_Weekly_weeks

,WeekNumber,Year,WeekStartDate,WeekEndDate
0,19,2024,2024-05-06,2024-05-12
1,20,2024,2024-05-13,2024-05-19
2,21,2024,2024-05-20,2024-05-26
3,22,2024,2024-05-27,2024-06-02


In [35]:
Output_Weekly_df.to_csv("Output_weekly.csv",index=False)